In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score, recall_score

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process02', sep=',')
# 如果有某值为nan，进行删除操作
df.dropna(axis=0, how='any', inplace=True)
df.head(10)

,label,from_163,from_tsinghua,from_126,from_yahoo,from_21cn,from_tom,from_12,from_cernet,from_sohu,from_unknown,has_not_date,jieba_cut_content,content_length_type_0,content_length_type_1,content_length_type_2,content_length_type_3,content_sema
0,1.0,0,0,0,0,0,0,0,0,0,1,0,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,0,0,1,0,8.456151
1,0.0,0,0,0,0,0,0,0,0,0,0,0,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,0,1,0,0,7.486084
2,1.0,0,0,0,0,0,0,0,0,0,0,0,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,0,1,0,0,7.175171
3,1.0,0,0,0,0,0,0,1,0,0,0,0,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,0,1,0,0,7.565682
4,1.0,0,0,0,0,0,0,0,0,0,0,0,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,0,0,0,1,2.063409
5,1.0,1,0,0,0,0,0,0,0,0,0,0,TO ： 贵 公司 经理 、 财务 您好 ！ 深圳市 春洋 贸易 有...,0,1,0,0,7.143747
6,0.0,0,0,0,0,0,0,0,0,0,0,0,那 他 为什么 不 愿意 起诉 ， 既然 这样 了 ！ 起诉 后 ...,0,1,0,0,4.807568
7,1.0,0,0,0,0,0,1,0,0,0,0,0,尊敬 的 负责人 （ 经理 ／ 财务 ） ： 您好 ！ 我 是 深...,0,1,0,0,6.593684
8,1.0,0,0,0,0,0,0,0,0,0,0,0,您好 以下 是 特别 为 阁下 发 的 香港 信息 ( 图片 ...,0,1,0,0,7.611074
9,0.0,0,0,0,0,0,0,0,0,0,0,0,我 觉得 ， 负债 不要紧 ， 最 重要 的 是 能 负得起 这个 ...,0,1,0,0,7.041340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64284 entries, 0 to 64618
Data columns (total 18 columns):
label                    64284 non-null float64
from_163                 64284 non-null int64
from_tsinghua            64284 non-null int64
from_126                 64284 non-null int64
from_yahoo               64284 non-null int64
from_21cn                64284 non-null int64
from_tom                 64284 non-null int64
from_12                  64284 non-null int64
from_cernet              64284 non-null int64
from_sohu                64284 non-null int64
from_unknown             64284 non-null int64
has_not_date             64284 non-null int64
jieba_cut_content        64284 non-null object
content_length_type_0    64284 non-null int64
content_length_type_1    64284 non-null int64
content_length_type_2    64284 non-null int64
content_length_type_3    64284 non-null int64
content_sema             64284 non-null float64
dtypes: float64(2), int64(15), object(1)
memory usage: 9.3

In [6]:
# 2. 数据分割
x_train, x_test, y_train, y_test = train_test_split(df[['has_not_date','jieba_cut_content','content_sema']], df['label'], test_size=0.2, random_state=0)
print("训练数据集大小:%d" % x_train.shape[0])
print("测试数据集大小:%d" % x_test.shape[0])

训练数据集大小:51427
测试数据集大小:12857


In [7]:
x_train.head(5)

,has_not_date,jieba_cut_content,content_sema
18809,0,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.849239
61631,0,声音 变换器 ： 适用 于 不同 型号 的 手机 ， 电话 ， 网络...,7.242064
5822,0,http : / / tengyingge . blogchina . co...,8.289843
5881,0,刚 开始 ， 是 会 有点 不 习惯 的 。 慢慢来 。 ps ：...,7.063309
26337,0,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.700159


In [8]:
# 3. 开始模型训练
# 3.1 特征工程，将文本数据转换为数值型数据
transfromer = TfidfVectorizer(norm='l2', use_idf=True)
svd = TruncatedSVD(n_components=20) # 因为TF转换之后的数据是特别大的，所以这里进行一个svd降维；选择20个特征
jieba_cut_content = list(x_train['jieba_cut_content'].astype('str')) # 获取列信息
transfromer_model = transfromer.fit(jieba_cut_content)
df1 = transfromer_model.transform(jieba_cut_content)
svd_model = svd.fit(df1)
df2 = svd_model.transform(df1)

data = pd.DataFrame(df2)
print(data.head(5))
print(data.info())

         0         1         2         3         4         5         6   \
0  0.340289  0.220013 -0.237479 -0.039944 -0.070173  0.002533 -0.160633   
1  0.014968  0.064739  0.026998 -0.004683  0.008163  0.007493  0.016363   
2  0.011862  0.072033  0.047005  0.001416 -0.002091  0.035848  0.009176   
3  0.019714  0.117438  0.132448  0.060408 -0.103484 -0.074794 -0.012163   
4  0.346257  0.215338 -0.234032 -0.043125 -0.071085 -0.000014 -0.162462   

         7         8         9         10        11        12        13  \
0 -0.010108  0.201798 -0.010479 -0.199859  0.038598 -0.065275  0.430410   
1  0.046169  0.029175 -0.000580  0.007718  0.023375  0.041320 -0.012053   
2 -0.012101 -0.016757  0.000156  0.001108 -0.004167 -0.002390 -0.003409   
3 -0.005835 -0.027584  0.000536 -0.016853 -0.013104 -0.003811  0.017763   
4 -0.009371  0.199050 -0.010354 -0.195497  0.036800 -0.061821  0.418546   

         14        15        16        17        18        19  
0 -0.481458  0.207269  0.091776 -0

In [9]:
# 3.2 数据合并
data['has_not_date'] = list(x_train['has_not_date'])
data['content_sema'] = list(x_train['content_sema'])
print(data.head(5))
print(data.info())

          0         1         2         3         4         5         6  \
0  0.340289  0.220013 -0.237479 -0.039944 -0.070173  0.002533 -0.160633   
1  0.014968  0.064739  0.026998 -0.004683  0.008163  0.007493  0.016363   
2  0.011862  0.072033  0.047005  0.001416 -0.002091  0.035848  0.009176   
3  0.019714  0.117438  0.132448  0.060408 -0.103484 -0.074794 -0.012163   
4  0.346257  0.215338 -0.234032 -0.043125 -0.071085 -0.000014 -0.162462   

          7         8         9      ...             12        13        14  \
0 -0.010108  0.201798 -0.010479      ...      -0.065275  0.430410 -0.481458   
1  0.046169  0.029175 -0.000580      ...       0.041320 -0.012053  0.012463   
2 -0.012101 -0.016757  0.000156      ...      -0.002390 -0.003409 -0.003590   
3 -0.005835 -0.027584  0.000536      ...      -0.003811  0.017763  0.002992   
4 -0.009371  0.199050 -0.010354      ...      -0.061821  0.418546 -0.471732   

         15        16        17        18        19  has_not_date  \
0  0.

In [10]:
%%time
nb = BernoulliNB(alpha=1.0,binarize=0.0005)
model = nb.fit(data, y_train)

Wall time: 66.1 ms


In [11]:
# 4.1 对测试数据进行特征转换
jieba_cut_content_test = list(x_test['jieba_cut_content'].astype('str'))
data_test = pd.DataFrame(svd_model.transform(transfromer_model.transform(jieba_cut_content_test)))
data_test['has_not_date'] = list(x_test['has_not_date'])
data_test['content_sema'] = list(x_test['content_sema'])
print(data_test.head(5))
print(data_test.info())
# 4.2 对测试数据进行预测
y_predict = model.predict(data_test)
y_predict

          0         1         2         3         4         5         6  \
0  0.537283  0.009055 -0.087589  0.028968  0.000887 -0.032611 -0.157830   
1  0.210902  0.131677 -0.090073 -0.086956 -0.033314  0.039730  0.006881   
2  0.045151  0.277282  0.233906  0.051034 -0.074005 -0.016294  0.010208   
3  0.032907  0.163067  0.113489  0.002456  0.053736  0.008293  0.004149   
4  0.035278  0.065205  0.026085 -0.023741  0.072929 -0.005900 -0.003348   

          7         8         9      ...             12        13        14  \
0  0.009262 -0.134854  0.009138      ...       0.078211  0.003356 -0.016100   
1 -0.008323  0.011483 -0.000151      ...      -0.026096  0.081973 -0.057208   
2 -0.014860  0.099690 -0.002227      ...      -0.069410 -0.040477 -0.030488   
3 -0.020261  0.117595 -0.002779      ...      -0.062614 -0.059600 -0.012782   
4 -0.008196  0.008662  0.001008      ...      -0.000170 -0.005765 -0.004045   

         15        16        17        18        19  has_not_date  \
0 -0.

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [12]:
# 5. 效果评估
print("精确率为:%.5f" % precision_score(y_test, y_predict))
print("召回率为:%.5f" % recall_score(y_test, y_predict))
print("F1值为:%.5f" % f1_score(y_test, y_predict))

准确率为:0.94643
召回率为:0.98878
F1值为:0.96714
